## Thanks to Lauren alexandra whose code i've copied to help guide me through the rough spots

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pathlib
import zipfile
from glob import glob
import tqdm as notebook_tqdm

import pandas as pd
import numpy as np
import geopandas as gpd
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import xarray as xr
import xrspatial

import matplotlib.pyplot as plt
import hvplot.pandas
import hvplot.xarray
import cartopy.crs as ccrs
import geoviews as gv




In [10]:
%store -r

In [ ]:
# Set Paths

# Define and create the project data directories
data_dir = os.path.join(
    pathlib.Path.home(),
    'Documents',
    'eaclassprojects',
    'climate_models',
)
os.makedirs(data_dir, exist_ok=True)
pathlib.Path.home()


# Datasets
model_data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    'Documents',
    'eaclassprojects',
    'model_data'
)

os.makedirs(model_data_dir, exist_ok=True)



In [ ]:
# def get_projected_climate(site_name, site_gdf,
#                           emissions_scenario, gcm, time_slices):
#     """
#     Create DataFrame of projected site climate for a given time period.
    
#     Args:
#     site_name (str): Site name.
#     site_gdf (dict): Site GeoDataFrame.
#     emissions_scenario (str): Climate scenario. 
#     gcm (str): Global Climate Model. 
#     time_slices (list): List of years indicating a time slice.
#     Returns:
#     pandas.DataFrame: A DataFrame of projected average max temperatures.
#     """

#     maca_das = []

    # for start_year in time_slices:
    #     end_year = start_year + 4

    #     # Multivariate Adaptive Constructed Analogs (MACA)
    #     MACA_URL = (
    #         'http://thredds.northwestknowledge.net:8080/thredds/fileServer/' 
    #         f'MACAV2/{gcm}/macav2metdata_tasmax_{gcm}_r1i1p1_'
    #         f'{emissions_scenario}_{start_year}_{end_year}_CONUS_monthly.nc'
    #     )

    #     maca_da = xr.open_dataset(MACA_URL, engine='h5netcdf').squeeze()

    #     bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds

    #     # update coordinate range
    #     maca_da = maca_da.assign_coords(
    #         lon=("lon", [convert_longitude(l) for l in maca_da.lon.values])
    #     )

    #     maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    #     maca_da = maca_da.rio.clip_box(*bounds)

    #     maca_das.append(dict(
    #                         site_name=site_name,
    #                         gcm=gcm,
    #                         start_year=start_year,
    #                         end_year=end_year,
    #                         da=maca_da))

    # return pd.DataFrame(maca_das)

def download_climate(site_name, site_gdf, emissions_scenario, 
                     climate_models, time_slices, raster_path, data_dir):
    """
    Retrieve projected site climate for a given time period, 
    build composite DataArray, and export raster.
    
    Args:
    site_name (str): Site name.
    site_gdf (dict): Site GeoDataFrame.
    emissions_scenario (str): Climate scenario. 
    climate_models (list): Climate model names.
    time_slices (list): List of years indicating a time slice.
    raster_path (str): Path of site climate raster.
    data_dir (str): Path of data directory.
    Returns: None
    """

    for gcm in climate_models:
        print(f'Downloading climate data for {gcm}')

        # Retrieve MACA climate data
        site_proj_temp = get_projected_climate(
                                        site_name, site_gdf, 
                                        emissions_scenario, gcm, 
                                        time_slices
                                    )

        # Convert temperature to fahrenheit 
        projected_climates_df = site_proj_temp.assign(
                                    fahrenheit=lambda x: x.da.map(
                                        convert_temperature))

        # Create composite site projected climate
        site_clim_composite_ds = xr.concat(
            projected_climates_df.fahrenheit, dim='time').mean('time')

        site_clim_composite_da = site_clim_composite_ds.to_dataarray(
                                    dim='air_temperature', 
                                    name='air_temperature')

        export_raster(site_clim_composite_da, 
                    f"{raster_path}_{gcm}_max_temp.tif", data_dir)



In [ ]:
# site dictionary
location_dict = {
    'wv': wayne_county,
    'nc': jackson_county
}

# Set climate parameters
hist_scenario = 'historical'
proj_scenario = 'rcp85'
climate_models = ['MIROC5', 'NorESMI-M', 'IPSL-CM5A-MR', 'GFDL-ESM2M'] 
last_century = [1970, 1975, 1980, 1985, 1990, 1995] # 1970-2000 
this_century = [2036, 2041, 2046, 2051, 2056, 2061] # 2036-2066
time_periods = [this_century, last_century]

# For historical climate data at NC location
download_climate('nc', jackson_county,
                hist_scenario, climate_models, last_century,
                'jackson_hist', model_data_dir)

# For projected climate data at NC location
download_climate('nc', jackson_county,
                proj_scenario, climate_models, this_century,
                'jackson_proj', model_data_dir)

# For historical climate data at WV location
download_climate('wv', wayne_county,
                hist_scenario, climate_models, last_century,
                'wayne_hist', model_data_dir)

# For projected climate data at WV location
download_climate('wv', wayne_county,
                proj_scenario, climate_models, this_century,
                'wayne_proj', model_data_dir)



NameError: name 'download_climate' is not defined

In [ ]:
# CanESM2 (Warm and wet)

build_habitat_suitability_model('eldorado', 'mid_century', 'CanESM2',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_mc_CanESM2_suitability')

build_habitat_suitability_model('eldorado', 'late_century', 'CanESM2',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_lc_CanESM2_suitability')

# MIROC-ESM-CHEM (Warm and dry)

build_habitat_suitability_model('eldorado', 'mid_century', 'MIROC-ESM-CHEM',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_mc_MIROC-ESM-CHEM_suitability')

build_habitat_suitability_model('eldorado', 'late_century', 'MIROC-ESM-CHEM',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_lc_MIROC-ESM-CHEM_suitability')

# CNRM-CM5 (Cool and wet)

build_habitat_suitability_model('eldorado', 'mid_century', 'CNRM-CM5',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_mc_CNRM-CM5_suitability')

build_habitat_suitability_model('eldorado', 'late_century', 'CNRM-CM5',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_lc_CNRM-CM5_suitability')

# GFDL-ESM2M (Cool and dry)

build_habitat_suitability_model('eldorado', 'mid_century', 'GFDL-ESM2M',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_mc_GFDL-ESM2M_suitability')

build_habitat_suitability_model('eldorado', 'late_century', 'GFDL-ESM2M',
                                optimal_values, tolerance_ranges, data_dir, 
                                'eld_lc_GFDL-ESM2M_suitability')

# %% [markdown]
# CanESM2 (Warm and wet)

# %%
plot_site(
    f"{data_dir}/eld_mc_CanESM2_suitability.tif", 
    eldorado_gdf, plots_dir,
    'Eldorado-National-Forest-Suitability-Mid-Century-CanESM2', 
    'Eldorado Mid Century Suitability: RCP 4.5 (CanESM2)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

plot_site(
    f"{data_dir}/eld_lc_CanESM2_suitability.tif", 
    eldorado_gdf, plots_dir,
    'Eldorado-National-Forest-Suitability-Late-Century-CanESM2', 
    'Eldorado Late Century Suitability: RCP 4.5 (CanESM2)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# MIROC-ESM-CHEM (Warm and dry)

# %%
plot_site(
    f"{data_dir}/eld_lc_MIROC-ESM-CHEM_suitability.tif",
    eldorado_gdf, plots_dir,
    'Eldorado-National-Forest-Suitability-Late-Century-MIROC-ESM-CHEM', 
    'Eldorado Late Century Suitability: RCP 4.5 (MIROC-ESM-CHEM)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# CNRM-CM5 (Cool and wet)

# %%
plot_site(
    f"{data_dir}/eld_lc_CNRM-CM5_suitability.tif",
    eldorado_gdf, plots_dir,
    'Eldorado-National-Forest-Suitability-Late-Century-CNRM-CM5', 
    'Eldorado Late Century Suitability: RCP 4.5 (CNRM-CM5)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# GFDL-ESM2M (Cool and dry)

# %%
plot_site(
    f"{data_dir}/eld_lc_GFDL-ESM2M_suitability.tif",
    eldorado_gdf, plots_dir,
    'Eldorado-National-Forest-Suitability-Late-Century-GFDL-ESM2M', 
    'Eldorado Late Century Suitability: RCP 4.5 (GFDL-ESM2M)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# #### Los Padres Suitability

# %%
# CanESM2 (Warm and wet)

build_habitat_suitability_model('los_padres', 'mid_century', 'CanESM2',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_mc_CanESM2_suitability')

build_habitat_suitability_model('los_padres', 'late_century', 'CanESM2',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_lc_CanESM2_suitability')

# MIROC-ESM-CHEM (Warm and dry)

build_habitat_suitability_model('los_padres', 'mid_century', 'MIROC-ESM-CHEM',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_mc_MIROC-ESM-CHEM_suitability')

build_habitat_suitability_model('los_padres', 'late_century', 'MIROC-ESM-CHEM',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_lc_MIROC-ESM-CHEM_suitability')

# CNRM-CM5 (Cool and wet)

build_habitat_suitability_model('los_padres', 'mid_century', 'CNRM-CM5',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_mc_CNRM-CM5_suitability')

build_habitat_suitability_model('los_padres', 'late_century', 'CNRM-CM5',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_lc_CNRM-CM5_suitability')

# GFDL-ESM2M (Cool and dry)

build_habitat_suitability_model('los_padres', 'mid_century', 'GFDL-ESM2M',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_mc_GFDL-ESM2M_suitability')

build_habitat_suitability_model('los_padres', 'late_century', 'GFDL-ESM2M',
                                optimal_values, tolerance_ranges, data_dir, 
                                'lp_lc_GFDL-ESM2M_suitability')

# %% [markdown]
# CanESM2 (Warm and wet)

# %%
plot_site(
    f"{data_dir}/lp_mc_CanESM2_suitability.tif", 
    los_padres_gdf, plots_dir,
    'Los-Padres-National-Forest-Suitability-Mid-Century-CanESM2', 
    'Los Padres Mid Century Suitability: RCP 4.5 (CanESM2)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

plot_site(
    f"{data_dir}/lp_lc_CanESM2_suitability.tif", 
    los_padres_gdf, plots_dir,
    'Los-Padres-National-Forest-Suitability-Late-Century-CanESM2', 
    'Los Padres Late Century Suitability: RCP 4.5 (CanESM2)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# MIROC-ESM-CHEM (Warm and dry)

# %%
plot_site(
    f"{data_dir}/lp_lc_MIROC-ESM-CHEM_suitability.tif", 
    los_padres_gdf, plots_dir,
    'Los-Padres-National-Forest-Suitability-Late-Century-MIROC-ESM-CHEM', 
    'Los Padres Late Century Suitability: RCP 4.5 (MIROC-ESM-CHEM)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# CNRM-CM5 (Cool and wet)

# %%
plot_site(
    f"{data_dir}/lp_lc_CNRM-CM5_suitability.tif", 
    los_padres_gdf, plots_dir,
    'Los-Padres-National-Forest-Suitability-Late-Century-CNRM-CM5', 
    'Los Padres Late Century Suitability: RCP 4.5 (CNRM-CM5)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)

# %% [markdown]
# GFDL-ESM2M (Cool and dry)

# %%
plot_site(
    f"{data_dir}/lp_lc_GFDL-ESM2M_suitability.tif", 
    los_padres_gdf, plots_dir,
    'Los-Padres-National-Forest-Suitability-Late-Century-GFDL-ESM2M', 
    'Los Padres Late Century Suitability: RCP 4.5 (GFDL-ESM2M)', 
    'Suitability', 'YlGn', 'black', tif_file=True
)


In [ ]:
### Data Wrangling ###

def build_da(urls, bounds):
    """
    Build a DataArray from a list of urls.
    
    Args:
    urls (list): Input list of URLs.
    bounds (tuple): Site boundaries.
    Returns:
    xarray.DataArray: A merged DataArray.
    """

    all_das = []

    # Add buffer to bounds for plotting
    buffer = .025
    xmin, ymin, xmax, ymax = bounds
    bounds_buffer = (xmin-buffer, ymin-buffer, xmax+buffer, ymax+buffer)

    for url in urls:
        # Open data granule, mask missing data, scale data, 
        # and remove dimensions of length 1
        tile_da = rxr.open_rasterio(
                url,
                # For the fill/missing value
                mask_and_scale=True
            ).squeeze()
        # Unpack the bounds and crop tile
        cropped_da = tile_da.rio.clip_box(*bounds_buffer)
        all_das.append(cropped_da)

    merged = merge_arrays(all_das)
    return merged

def export_raster(da, raster_path, data_dir):
    """
    Export raster DataArray to a raster file.
    
    Args:
    raster (xarray.DataArray): Input raster layer.
    raster_path (str): Output raster directory.
    data_dir (str): Path of data directory.
    Returns: None
    """

    output_file = os.path.join(data_dir, os.path.basename(raster_path))
    da.rio.to_raster(output_file)

def harmonize_raster_layers(reference_raster, input_rasters, data_dir):
    """
    Harmonize raster layers to ensure consistent spatial resolution 
    and projection.
    Args:
    reference_raster (str): Path of raster to reference.
    input_rasters (list): List of rasters to harmonize.
    data_dir (str): Path of data directory.
    Returns:
    list: A list of harmonized rasters.
    """

    harmonized_files = []

    harmonized_files.append(reference_raster)
    # Load the reference raster
    ref_raster = rxr.open_rasterio(reference_raster, masked=True)
    # Use projection EPSG:3857 (WGS 84 with x/y coords)
    ref_raster = ref_raster.rio.write_crs(3857)

    for raster_path in input_rasters:
        # Load the input raster
        input_raster = rxr.open_rasterio(raster_path, masked=True)
        input_raster = input_raster.rio.write_crs(3857)

        # Reproject and align the input raster to match the reference raster

        # Only 2D/3D arrays with dimensions x/y are currently supported
        # by reproject_match()
        harmonized_raster = input_raster.rio.reproject_match(ref_raster)

        # Save the harmonized raster to the output directory
        output_file = os.path.join(data_dir, os.path.basename(raster_path))
        harmonized_raster.rio.to_raster(output_file)
        harmonized_files.append(output_file)

    return harmonized_files

### Calculations ###

def calculate_suitability_score(raster, optimal_value, tolerance_range):
    """ 
    Calculate a fuzzy suitability score (0–1) for each raster cell based on 
    proximity to the optimal value.
    Args:
    raster (xarray.DataArray): Input raster layer. 
    optimal_value (float): Optimal value for the variable.
    tolerance_range (float): Suitable values range. 
    Returns:
    xarray.DataArray: A raster of suitability scores (0-1).
    """

    # Calculate using a fuzzy Gaussian function to assign scores 
    # between 0 and 1
    suitability = np.exp(
                    -((raster - optimal_value) ** 2) 
                    / (2 * tolerance_range ** 2)
                )

    # Suitability scores (0–1) 
    return suitability

### Visualization ###



    harmonized_rasters = harmonize_raster_layers(reference_raster, 
                                                 input_rasters, data_dir)

    # Load and calculate suitability scores for each raster
    suitability_layers = []
    suit_zip = zip(harmonized_rasters, optimal_values, tolerance_ranges)
    for raster_path, optimal_value, tolerance_range in suit_zip:
        raster = rxr.open_rasterio(raster_path, masked=True).squeeze()
        suitability_layer = calculate_suitability_score(
                                raster, optimal_value, tolerance_range
                            )
        suitability_layers.append(suitability_layer)

    # Combine suitability scores by multiplying across all layers
    combined_suitability = suitability_layers[0]
    for layer in suitability_layers[1:]:
        combined_suitability *= layer

    # Save the combined suitability raster
    output_file = os.path.join(data_dir, f"{raster_name}.tif")
    combined_suitability.rio.to_raster(output_file)
    print(f"Combined suitability raster saved to: {raster_name}.tif")

    # Path to the final combined suitability raster
    return output_file

In [ ]:
### Create Suitability Model ###

def build_habitat_suitability_model(site_name, time_period, gcm,
                                    optimal_values, tolerance_ranges, 
                                    data_dir, raster_name):
    """
    Build a habitat suitability model by combining fuzzy suitability scores 
    for each variable. 
    Args:
    site_name (str): Name of site.
    time_period (str): Name of time period. 
    gcm (str): Global Climate Model.    
    optimal_values (list): List of optimal values for variables.
    tolerance_ranges (list): List of tolerance values for variables.
    data_dir (str): Path of data directory.
    raster_name (str): The name of model raster.
    Returns:
    str: The path of the suitability raster.
    """
    reference_raster = f"{data_dir}/{site_name}_elevation.tif" 

    input_rasters = [
        f"{data_dir}/{site_name}_{time_period}_{gcm}_max_temp.tif",
        f"{data_dir}/{site_name}_aspect.tif",
        f"{data_dir}/{site_name}_soil_ph.tif"
    ]
